<a href="https://colab.research.google.com/github/kochhar/keras-word2vec/blob/master/Keras_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://kochhar@github.com/kochhar/keras-word2vec

Cloning into 'keras-word2vec'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 45 (delta 17), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [2]:
import os
os.chdir('/content/keras-word2vec')
os.getcwd()

'/content/keras-word2vec'

In [159]:
!git remote set-url origin https://kochhar@github.com/kochhar/keras-word2vec
!git pull

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/kochhar/keras-word2vec
   13fe8f4..f96f1ea  master     -> origin/master
hint: Waiting for your editor to close the file... fatal: cannot run editor: No such file or directory
error: unable to start editor 'editor'
Not committing merge; use 'git commit' to complete the merge.


In [36]:
import collections
import logging
import numpy as np
from sklearn.model_selection import train_test_split
import word2vec as w2v
from importlib import reload
w2v = reload(w2v)

logging.basicConfig(format="[%(asctime)s]%(levelname)s:%(name)s:%(message)s",
                    level=logging.INFO)
logging.info('logging configured')


[2018-10-30 03:26:30,161]INFO:root:logging configured


In [15]:
filename = 'data/text8.zip'
logging.info('Reading corpus')
training_corpus = w2v.str_from_zipfile(filename).split()
logging.info('Corpus: %s words', len(training_corpus))

[2018-10-30 03:07:14,666]INFO:root:Reading corpus
[2018-10-30 03:07:16,853]INFO:root:Corpus: 17005207 words


In [16]:
logging.info('Creating vocabulary')
counts = [('UNK', -1)] + collections.Counter(training_corpus).most_common(int(1e4-1))
vocab = w2v.Vocabulary([word for (word, _) in counts ])
logging.info('Encoding corpus')
tokens = vocab.encode(training_corpus)
logging.info("Vocab: %s tokens, Sample: %s", vocab.size, list(zip(training_corpus[:7], tokens[:7])))
del training_corpus


[2018-10-30 03:07:20,751]INFO:root:Creating vocabulary
[2018-10-30 03:07:24,523]INFO:root:Encoding corpus
[2018-10-30 03:07:27,941]INFO:root:Vocab: 10000 tokens, Sample: [('anarchism', 5234), ('originated', 3081), ('as', 12), ('a', 6), ('term', 195), ('of', 2), ('abuse', 3134)]


In [18]:
logging.info('Creating skipgrams')
word_target, word_context, labels = sgns.skipgrams(tokens, window_size=3)
logging.info('Training skipgrams: %s, Sample: %s', len(word_target), 
             list(zip(word_target[:10], word_context[:10], labels[:10])))
del tokens

[2018-10-30 03:07:38,411]INFO:root:Creating skipgrams
[2018-10-30 03:12:05,531]INFO:root:Training skipgrams: 30022202, Sample: [(9817, 3, 1), (1460, 1264, 0), (6786, 831, 0), (6777, 7884, 0), (278, 990, 0), (606, 6433, 0), (5230, 8627, 0), (669, 9626, 0), (1057, 1614, 0), (2831, 4311, 0)]


In [0]:
class Similarity(object):
    def __init__(self, sim_model):
        """Params:
            - sim_model: A keras model which outputs a similarity score given
              two vectors as inputs"""
        self.sim_model = sim_model

    def most_similar(self, examples, vocab, top_k=10):
        sim_fn = self.sim
        contexts = range(vocab.size)
        # compute the similarity between the example and every context word in the vocabulary
        sims = [(ex, sim_fn(ex, contexts)) for ex in examples]
        sims = [(ex, (-sim).argsort()[1:top_k+1], sim) for (ex, sim) in sims]
        return [(ex, top_idx, sim[top_idx]) for (ex, top_idx, sim) in sims]

    def sim(self, target, contexts):
        context_b = np.array(contexts)
        # the target word for every item in the batch is the same
        target_b = np.array([target]*len(contexts))
        sim_b = self.sim_model.predict_on_batch([target_b, context_b])
        
        # reshape from (len(contexts), 1) to (len(contexts),)
        sim_b = sim_b.reshape(context_b.shape)
        return sim_b


In [100]:
# Build the model
sgns = w2v.SkipGramNegSample(vocab_size=vocab.size, embedding_dim=300)
model, validation_model = sgns.model()
sim_eval = Similarity(validation_model)

# Validation set for inspecting training progress
# Max index of the random sampling from tokens (implies choosing
# from the the top-N occurring tokens)
max_token_idx = 128
valid_samples = np.random.choice(np.arange(24, max_token_idx), size=16, replace=False)

# Feeding to the network
batch_size = 128
tgt_b, ctx_b, lbl_b = np.zeros((batch_size,)), np.zeros((batch_size,)), np.zeros((batch_size,))
# training loop over 1e6 epochs with batches of size 32
for epoch in range(6):
  for idx in range(0, 1024000, batch_size):
    offset = epoch * 1024000 + idx
    tgt_b[0:batch_size,] = word_target[offset:offset+batch_size]
    ctx_b[0:batch_size,] = word_context[offset:offset+batch_size]
    lbl_b[0:batch_size,] = labels[offset:offset+batch_size]
    loss = model.train_on_batch([tgt_b, ctx_b], lbl_b)

    if idx % 51200 == 0:
      logging.info("Epoch: %s, example@%s, loss=%s", epoch, offset, loss)
  
  most_similar = sim_eval.most_similar(valid_samples, vocab)
  log_str = '\n'+'\n'.join(['Most similar to {} (median: {:.5}): {}' .format(vocab.decode([ex]), scores[4], 
                                                                             ', '.join(vocab.decode(top_idx)))
                           for (ex, top_idx, scores) in most_similar])
  logging.info(log_str)


[2018-10-30 05:30:50,431]INFO:root:Epoch: 0, example@0, loss=0.6927639
[2018-10-30 05:30:53,555]INFO:root:Epoch: 0, example@51200, loss=0.69472253
[2018-10-30 05:30:56,677]INFO:root:Epoch: 0, example@102400, loss=0.69249415
[2018-10-30 05:30:59,771]INFO:root:Epoch: 0, example@153600, loss=0.6925862
[2018-10-30 05:31:02,869]INFO:root:Epoch: 0, example@204800, loss=0.6883968
[2018-10-30 05:31:05,941]INFO:root:Epoch: 0, example@256000, loss=0.67755044
[2018-10-30 05:31:09,035]INFO:root:Epoch: 0, example@307200, loss=0.65029216
[2018-10-30 05:31:12,120]INFO:root:Epoch: 0, example@358400, loss=0.78567576
[2018-10-30 05:31:15,182]INFO:root:Epoch: 0, example@409600, loss=0.58435905
[2018-10-30 05:31:18,268]INFO:root:Epoch: 0, example@460800, loss=0.7329979
[2018-10-30 05:31:21,354]INFO:root:Epoch: 0, example@512000, loss=0.57221025
[2018-10-30 05:31:24,451]INFO:root:Epoch: 0, example@563200, loss=0.6042657
[2018-10-30 05:31:27,555]INFO:root:Epoch: 0, example@614400, loss=0.5417385
[2018-10-30

In [148]:
embedding = model.get_layer('embedding')
# first set of weights are the learned embedding vectors
word_vectors = embedding.get_weights()[0]
logging.info('Learned %s %sD vectors', *learned_word_vectors.shape)
out = open('data/word2vec.txt', 'w')
out.write('{} {}\n'.format(vocab.size-1, sgns.embedding_dim))
for word, idx in vocab.word_2_idx.items():
  out.write('{} {}\n'.format(word, ' '.join(map(str, word_vectors[idx,]))))
out.close
logging.info('Wrote embedding weights to data/word2vec.txt')

[2018-10-30 07:08:33,881]INFO:root:Learned 10000 300D vectors
[2018-10-30 07:08:35,776]INFO:root:Wrote embedding weights to data word2vec.txt


In [150]:
!head data/word2vec.txt

9999 300
UNK 0.010102738 -0.015697192 0.024387922 4.082918e-05 -0.024800373 -0.014498748 0.037744883 0.007871889 0.011741649 0.039989617 -0.039880253 -0.027034689 0.0076042786 -0.024242878 -0.03025651 -0.014522016 -0.03167983 0.049274985 0.035965648 0.038065765 -0.038012553 -0.022853578 -0.0069702044 0.00016641617 0.03753606 0.021492492 -0.04014889 -0.0032878146 0.031786595 0.016863074 -0.033296444 -0.03644506 0.03576001 -0.0043700114 -0.022072423 -0.048637487 0.0283379 0.004434932 -0.00087534264 -0.022371018 0.0076481104 0.04931618 -0.000258971 0.035994325 0.026989128 0.0037972108 0.03734732 0.018676292 -0.0085910335 -0.024264503 0.016929273 0.035072695 -0.007653117 0.010401774 -0.018887663 0.035085406 0.04949807 -0.033115804 -0.025930513 0.013284732 0.029619489 -0.033128418 0.01675326 0.023451243 0.043576453 -0.01486541 -0.007862259 -0.028250301 0.028645765 -0.04517133 -0.03324919 -0.013556741 -0.01082753 0.04502252 0.016480032 -0.030329837 -0.048045065 -0.009424888 -0.034159996 0.01

In [154]:
!git config --global user.email "shailesh.kochhar@gmail.com"
!git config --global user.name "Shailesh Kochhar"
!git add data/word2vec.txt
!git commit -m "updated weights"

[master 180848f] updated weights
 1 file changed, 10000 insertions(+)
 create mode 100644 data/word2vec.txt


On branch master
Your branch and 'origin/master' have diverged,
and have 1 and 1 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    Keras-Word2Vec.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Keras_Word2Vec.ipynb
	word2vec/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
